In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="1"
import sys
import random

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('seaborn-white')
import seaborn as sns
sns.set_style("white")

%matplotlib inline

from sklearn.model_selection import train_test_split

from tqdm import tqdm_notebook #, tnrange
#from itertools import chain
from skimage.io import imread, imshow #, concatenate_images
from skimage.transform import resize
from skimage.morphology import label

import keras
from keras.models import Model, load_model, save_model
from keras.layers import Input,Dropout,BatchNormalization,Activation,Add, Dense, Input, Dropout
from keras.layers.core import Lambda
from keras.layers.convolutional import Conv2D, Conv2DTranspose
from keras.layers.pooling import MaxPooling2D
from keras.layers.merge import concatenate
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from keras import backend as K
from keras import optimizers

import tensorflow as tf

from keras.preprocessing.image import array_to_img, img_to_array, load_img#,save_img

import time
from kaggle_util import *
from models import *

from albumentations import (
    PadIfNeeded,
    HorizontalFlip,
    VerticalFlip,    
    CenterCrop,    
    Crop,
    Compose,
    Transpose,
    RandomRotate90,
    ElasticTransform,
    GridDistortion, 
    OpticalDistortion,
    RandomSizedCrop,
    OneOf,
    CLAHE,
    RandomContrast,
    RandomGamma,
    RandomBrightness
)


t_start = time.time()

Using TensorFlow backend.


In [2]:
img_size_ori = 101
img_size_target = 101

def auc(y_true, y_pred):
    auc = tf.metrics.auc(y_true, y_pred)[1]
    K.get_session().run(tf.local_variables_initializer())
    return auc

def upsample(img):
    if img_size_ori == img_size_target:
        return img
    return resize(img, (img_size_target, img_size_target), mode='constant', preserve_range=True)
    
def downsample(img):
    if img_size_ori == img_size_target:
        return img
    return resize(img, (img_size_ori, img_size_ori), mode='constant', preserve_range=True)

In [3]:
# Loading of training/testing ids and depths
train_df = pd.read_csv("../input/train.csv", index_col="id", usecols=[0])
depths_df = pd.read_csv("../input/depths.csv", index_col="id")
train_df = train_df.join(depths_df)
test_df = depths_df[~depths_df.index.isin(train_df.index)]

len(train_df)

train_df["images"] = [np.array(load_img("../input/train/images/{}.png".format(idx), grayscale=False)) / 255 for idx in tqdm_notebook(train_df.index)]
train_df["masks"] = [np.array(load_img("../input/train/masks/{}.png".format(idx), grayscale=True)) / 255 for idx in tqdm_notebook(train_df.index)]
train_df["coverage"] = train_df.masks.map(np.sum) / pow(img_size_ori, 2)
train_df["coverage_class"] = train_df.coverage.map(cov_to_class)
train_df['hassalt'] = train_df['masks'].apply(lambda x: (x.max()!=0) * 1)

/home/kownse/.local/lib/python3.6/site-packages/keras_preprocessing/image.py:487: UserWarning: grayscale is deprecated. Please use color_mode = "grayscale"
  warnings.warn('grayscale is deprecated. Please use '


In [4]:
SUBSET = len(train_df)
train_df = train_df.head(SUBSET)
len(train_df)

4000

In [5]:
x_train, x_valid, msk_train, msk_val, y_train, y_valid, depth_train, depth_test = train_test_split(
np.array(train_df.images.map(upsample).tolist()).reshape(-1, img_size_target, img_size_target, 3), 
np.array(train_df.masks.map(upsample).tolist()).reshape(-1, img_size_target, img_size_target, 1), 
train_df.hassalt.values,
train_df.z.values,
test_size=0.2, stratify=train_df.coverage_class, random_state= 1234)

In [6]:
aug_img = []
aug_y = []
augments = [
    (0.5, HorizontalFlip(p=1)),
#     (0.2, VerticalFlip(p=1)),
#     (0.5, RandomRotate90(p=1)),
#     (0.5, Transpose(p=1)),
    (0.5, ElasticTransform(p=1, alpha=120, sigma=120 * 0.05, alpha_affine=120 * 0.03)),
    (0.5, GridDistortion(p=1)),
    (0.5, OpticalDistortion(p=1, distort_limit=2, shift_limit=0.5)),
#     (0.5, RandomSizedCrop(p=1, min_max_height=(int(img_size_ori / 2), img_size_ori), height=img_size_ori, width=img_size_ori)),
]

for ratio, aug in tqdm_notebook(augments):
    selidx = np.random.choice(x_train.shape[0], int(x_train.shape[0] * ratio), replace=False)
    for idx in tqdm_notebook(selidx):
        augmented = aug(image=x_train[idx], mask=msk_train[idx])
        aimg = augmented['image']
        if len(aimg.shape) < 3:
            aimg = aimg[...,np.newaxis]

        aug_img.append(aimg)
        aug_y.append(y_train[idx])

aug_img = np.asarray(aug_img)
aug_y = np.asarray(aug_y)
x_train = np.append(x_train, aug_img, axis=0)
y_train = np.append(y_train, aug_y, axis=0)
print(x_train.shape)
print(y_train.shape)


(9600, 101, 101, 3)
(9600,)


In [7]:
def get_model(BASE_MODEL):
    from keras.preprocessing.image import ImageDataGenerator
    if BASE_MODEL=='VGG16':
        from keras.applications.vgg16 import VGG16 as PTModel, preprocess_input
    elif BASE_MODEL=='RESNET52':
        from keras.applications.resnet50 import ResNet50 as PTModel, preprocess_input
    elif BASE_MODEL=='InceptionV3':
        from keras.applications.inception_v3 import InceptionV3 as PTModel, preprocess_input
    elif BASE_MODEL=='Xception':
        from keras.applications.xception import Xception as PTModel, preprocess_input
    elif BASE_MODEL=='DenseNet169': 
        from keras.applications.densenet import DenseNet169 as PTModel, preprocess_input
    elif BASE_MODEL=='DenseNet121':
        from keras.applications.densenet import DenseNet121 as PTModel, preprocess_input
    else:
        raise ValueError('Unknown model: {}'.format(BASE_MODEL))
        
    return PTModel, preprocess_input

In [8]:
def train_classifier(base_name = 'VGG16'):
    print(base_name)
    PTModel, preprocess_input = get_model(base_name)
    x_train_act = x_train.copy()
#     for idx in range(len(x_train_act)):
#         x_train_act[idx] = preprocess_input(x_train_act[idx])

    x_val_act = x_valid.copy()
#     for idx in range(len(x_val_act)):
#         x_val_act[idx] = preprocess_input(x_val_act[idx])

    inputshape = (img_size_target, img_size_target, 3)
    PTModel, preprocess_input = get_model('VGG16')
    base_pretrained_model = PTModel(input_shape = inputshape, 
                                  include_top = False, weights = 'imagenet')
    # base_pretrained_model.trainable = False

    from keras import models, layers
    from keras.optimizers import Adam
    img_in = layers.Input(inputshape, name='Image_RGB_In')
    x = base_pretrained_model(img_in)
    x = layers.BatchNormalization()(x)
    x = layers.GlobalMaxPooling2D()(x)

    x = Dense(256)(x)
    x = BatchActivate(x)
    x = Dropout(0.5)(x)

    x = Dense(64)(x)
    x = BatchActivate(x)
    x = Dropout(0.5)(x)

    out_layer = layers.Dense(1, activation = 'sigmoid')(x)
    class_model = models.Model(inputs = [img_in], outputs = [out_layer], name = 'full_model')

    class_model.compile(optimizer = Adam(lr=0.01), 
                       loss = 'binary_crossentropy',
                       metrics = ['binary_accuracy', my_iou_metric])

    batch_size = 32
    basic_name = '../model/classifier/{}_argu'.format(base_name)
    save_model_name = basic_name + '.model'
    submission_file = basic_name + '.csv'

    print(save_model_name)
    print(submission_file)


    board = keras.callbacks.TensorBoard(log_dir='log/classifier/{}_argu'.format(base_name),
                           histogram_freq=0, write_graph=True, write_images=False)
    early_stopping = EarlyStopping(monitor='val_binary_accuracy', mode = 'max',patience=6, verbose=1)
    model_checkpoint = ModelCheckpoint(save_model_name,monitor='val_binary_accuracy', 
                                       mode = 'max', save_best_only=True, verbose=1)
    reduce_lr = ReduceLROnPlateau(monitor='val_binary_accuracy', mode = 'max',factor=0.5, patience=2, min_lr=0.00001, verbose=1)

    epochs = 200

    history = class_model.fit(x_train, y_train,
                        validation_data=[x_valid, y_valid], 
                        epochs=epochs,
                        batch_size=batch_size,
                        callbacks=[board, early_stopping, model_checkpoint,reduce_lr], 
                        verbose=1)

In [9]:
model_lst = ['Xception', 'DenseNet169', 'DenseNet121']
for base_model in model_lst:
    train_classifier(base_model)

Xception
../model/classifier/Xception_argu.model
../model/classifier/Xception_argu.csv
Train on 9600 samples, validate on 800 samples
Epoch 1/200
9600/9600 [==============================] - 33s 3ms/step - loss: 0.6755 - binary_accuracy: 0.5965 - my_iou_metric: 0.5965 - val_loss: 0.6687 - val_binary_accuracy: 0.6100 - val_my_iou_metric: 0.6100

Epoch 00001: val_binary_accuracy improved from -inf to 0.61000, saving model to ../model/classifier/Xception_argu.model
Epoch 2/200
9600/9600 [==============================] - 30s 3ms/step - loss: 0.6124 - binary_accuracy: 0.6515 - my_iou_metric: 0.6515 - val_loss: 0.8523 - val_binary_accuracy: 0.6100 - val_my_iou_metric: 0.6100

Epoch 00002: val_binary_accuracy did not improve from 0.61000
Epoch 3/200
9600/9600 [==============================] - 30s 3ms/step - loss: 0.5480 - binary_accuracy: 0.7210 - my_iou_metric: 0.7210 - val_loss: 0.7773 - val_binary_accuracy: 0.6100 - val_my_iou_metric: 0.6100

Epoch 00003: val_binary_accuracy did not impr

KeyboardInterrupt: 

In [ ]:
# model = load_model(save_model_name,custom_objects={'my_iou_metric_2': my_iou_metric_2,
#                                                    'lovasz_loss': lovasz_loss})



In [ ]:
# preds_valid = predict_result(model,x_valid,img_size_target)
# ## Scoring for last model, choose threshold by validation data 
# thresholds_ori = np.linspace(0.3, 0.7, 31)
# # Reverse sigmoid function: Use code below because the  sigmoid activation was removed
# thresholds = np.log(thresholds_ori/(1-thresholds_ori)) 

# # ious = np.array([get_iou_vector(y_valid, preds_valid > threshold) for threshold in tqdm_notebook(thresholds)])
# # print(ious)
# ious = np.array([iou_metric_batch(y_valid, preds_valid > threshold) for threshold in tqdm_notebook(thresholds)])
# print(ious)

# # instead of using default 0 as threshold, use validation data to find the best threshold.
# threshold_best_index = np.argmax(ious) 
# iou_best = ious[threshold_best_index]
# threshold_best = thresholds[threshold_best_index]

# plt.plot(thresholds, ious)
# plt.plot(threshold_best, iou_best, "xr", label="Best threshold")
# plt.xlabel("Threshold")
# plt.ylabel("IoU")
# plt.title("Threshold vs IoU ({}, {})".format(threshold_best, iou_best))
# plt.legend()

In [ ]:
# x_test = np.array([(np.array(load_img("../input/test/images/{}.png".format(idx), grayscale = True))) / 255 for idx in tqdm_notebook(test_df.index)]).reshape(-1, img_size_target, img_size_target, 1)

# preds_test = predict_result(model,x_test,img_size_target)